In [ ]:
cd /content/drive/MyDrive/project/Dataset

/content/drive/MyDrive/project/Dataset


In [ ]:
import cv2,os
data_path='/content/drive/MyDrive/project/Dataset/'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]
label_dict=dict(zip(categories,labels))

In [ ]:
print(label_dict)
print(categories)
print(labels)

{'Acne': 0, 'Melanoma': 1, 'Psoriasis': 2}
['Acne', 'Melanoma', 'Psoriasis']
[0, 1, 2]


In [ ]:
img_size=245
data=[]
target=[]

for category in categories:
  folder_path=os.path.join(data_path,category)
  img_names=os.listdir(folder_path)

  for img_name in img_names:
      img_path=os.path.join(folder_path,img_name)
      img=cv2.imread(img_path)

      try:
          resized=cv2.resize(img,(img_size,img_size))
          data.append(resized)
          target.append(label_dict[category])
      except Exception as e:
        print('Exception:',e)


In [ ]:


import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)



Using TensorFlow backend.


In [ ]:
new_target.shape


(1397, 3)

In [ ]:
data.shape
print(data)

In [ ]:
data.shape[1:]

(245, 245, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),activation='relu',input_shape=(245,245,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3),activation='relu'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers
model.add(Conv2D(128,(3,3),activation='relu'))

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(512,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(3,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 243, 243, 200)     5600      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 121, 121, 200)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 119, 119, 100)     180100    
_________________________________________________________________
activation_1 (Activation)    (None, 119, 119, 100)     0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 59, 59, 100)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 57, 57, 128)       115328    
_________________________________________________________________
flatten_1 (Flatten)          (None, 415872)           

In [ ]:


from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)



In [ ]:


train_data.shape



(1257, 245, 245, 3)

In [ ]:


train_target.shape



(1257, 3)

In [ ]:

history=model.fit(train_data,train_target,epochs=75)

Epoch 1/75
1257/1257 [==============================] - 15s 12ms/step - loss: 0.0233 - accuracy: 0.9968
Epoch 2/75
1257/1257 [==============================] - 15s 12ms/step - loss: 0.0070 - accuracy: 0.9984
Epoch 3/75
1257/1257 [==============================] - 15s 12ms/step - loss: 0.0025 - accuracy: 0.9992
Epoch 4/75
1257/1257 [==============================] - 15s 12ms/step - loss: 7.8531e-04 - accuracy: 1.0000
Epoch 5/75
1257/1257 [==============================] - 15s 12ms/step - loss: 5.9797e-04 - accuracy: 1.0000
Epoch 6/75
1257/1257 [==============================] - 15s 12ms/step - loss: 1.4508e-04 - accuracy: 1.0000
Epoch 7/75
1257/1257 [==============================] - 15s 12ms/step - loss: 1.2250e-04 - accuracy: 1.0000
Epoch 8/75
1257/1257 [==============================] - 15s 12ms/step - loss: 1.0156e-04 - accuracy: 1.0000
Epoch 9/75
1257/1257 [==============================] - 15s 12ms/step - loss: 9.4072e-05 - accuracy: 1.0000
Epoch 10/75
1257/1257 [=================

In [ ]:
model.save('model.h5')